# Get Drive items

## Install and Import

In [1]:
%pip install pandas msal openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import msal
import requests
import pandas as pd

## Configs

### Skapa Service Principle
följ våran guid och skapa SP:n och fill i Cell:en nedan.
> notera att `CLIENT_ID` är `Application (client) ID`

In [3]:
TENANT_ID="8f4baa8a-c188-4891-8a8c-0b7d53418c93"
CLIENT_ID="1c0887a1-2b6c-4a95-9dfb-53901c3049a5"
CLIENT_SECRET="<secret>"

### Users
Add users

In [4]:
user_ids = {
  "bardia": "07b076e1-9273-44ab-afc2-4653303eb368",
  "AdeleV": "AdeleV@M365B806348.OnMicrosoft.com",
  "AlexW": "AlexW@M365B806348.OnMicrosoft.com",
  "AllanD": "AllanD@M365B806348.OnMicrosoft.com",
}

### functions

In [5]:
ms_graph_url = "https://graph.microsoft.com/v1.0"
SCOPE = ["https://graph.microsoft.com/.default"]


def get_headers():
    app = msal.ConfidentialClientApplication(
        CLIENT_ID,
        authority=f"https://login.microsoftonline.com/{TENANT_ID}",
        client_credential=CLIENT_SECRET,
    )
    r = app.acquire_token_for_client(SCOPE)
    access_token = r["access_token"]
    return {
        "Authorization": "Bearer " + access_token,
        "Content-Type": "application/json",
    }


def get_drive_id(user_id):
    url = ms_graph_url + f"/users/{user_id}/drive/"
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        raise f"call failed"
    return res.json()["id"]


def get_folder_items(user_id, folder_id="root"):
    url = ms_graph_url + f"/users/{user_id}/drive/items/{folder_id}/children"
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        raise f"call failed"
    return pd.DataFrame(res.json()["value"])


def get_folder_with_items(data: pd.DataFrame):
    folders = data[~data["folder"].isna()]
    return folders[[f["childCount"] > 0 for f in folders["folder"]]]


def get_folder_ids(data: pd.DataFrame) -> list[str]:
    try:
        return list(get_folder_with_items(data)["id"])
    except KeyError:
        return []


def get_files(data: pd.DataFrame) -> list[str]:
    try:
        return data[~data["file"].isna()]
    except KeyError:
        return pd.DataFrame()


def sort_items_folder(data: pd.DataFrame):
    return get_files(data), get_folder_ids(data)


def get_users_files(user_id):
    folders_ids = ["root"]
    items = pd.DataFrame()
    while len(folders_ids) > 0:
        curr_folder = folders_ids.pop()
        output = get_folder_items(user_id, curr_folder)
        curr_items, new_folder_ids = sort_items_folder(output)
        items = pd.concat([curr_items, items])
        folders_ids.extend(new_folder_ids)
    return items


headers = get_headers()

## Get report

In [9]:
user_id = user_ids['AdeleV']
items = get_users_files(user_id)
items[['id', 'name', 'folder', 'size', 'webUrl', 'file']]

,id,name,folder,size,webUrl,file
0,01NVXOCCYWY44RBZ2BMREJXMG7FXZ5EROG,Shelter Volunteers.xlsx,NaN,8596,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'e/qrPTn7GNxUx13dS...
1,01NVXOCCZRE5P4TZCFVFC3NJKFH5OPEGR3,New Product Presentation.pptx,NaN,491171,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'fAOB5W6oIVM6zU7jm...
2,01NVXOCC7NKNYK4CPXHNCZEKGABDF5VKEF,New Product Pricing.docx,NaN,729957,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'MuMwGRG5AeP2U1tVT...
3,01NVXOCCYHFWTXZLZM3RAZEFFZU2KAS2JT,QT1000 Marketing Plan.pptx,NaN,449499,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'N0BZOY6yYBUOb1K7m...
4,01NVXOCC5LAWEDKLVMIZEJXLUPOSTVJJWR,Timesheet_AdeleV.xlsx,NaN,12091,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'FX4t5RqUxVkEY6Wdy...
5,01NVXOCC7TQOI3EOSJQ5G3TEBSSDPCNKLF,ZT6006 Major Selling Points.docx,NaN,25340,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': '7f3F/6yD7rByFLu8v...


### Filter results

In [22]:
filtered_items = items[items['name'].str.contains('Product', case=True)]
filtered_items[['id', 'name', 'folder', 'size', 'webUrl', 'file']]

,id,name,folder,size,webUrl,file
1,01NVXOCCZRE5P4TZCFVFC3NJKFH5OPEGR3,New Product Presentation.pptx,NaN,491171,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'fAOB5W6oIVM6zU7jm...
2,01NVXOCC7NKNYK4CPXHNCZEKGABDF5VKEF,New Product Pricing.docx,NaN,729957,https://m365b806348-my.sharepoint.com/personal...,{'hashes': {'quickXorHash': 'MuMwGRG5AeP2U1tVT...


In [11]:
items.to_excel(f"{user_id}_files.xlsx")